In [3]:
!pip install py2neo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 2.7 MB/s eta 0:00:00a 0:00:01


In [4]:
from neo4j import GraphDatabase

In [6]:
# Neo4j connection parameters
uri = "bolt://localhost:7687"
user = "neo4j"
password = "12345678"

# Create driver
driver = GraphDatabase.driver(uri, auth=(user, password))

In [18]:
# Function to retrieve any User name
def get_any_user_name(tx):
    result = tx.run("MATCH (u:User) RETURN u.username LIMIT 1")
    for record in result:
        print("User name:", record["u.username"])

# Run query in a session
with driver.session() as session:
    session.read_transaction(get_any_user_name)

User name: William


/var/folders/mc/wkczfldx7tsb21fbkv6dnnjw0000gn/T/ipykernel_6289/324096550.py:9: DeprecationWarning: read_transaction has been renamed to execute_read
  session.read_transaction(get_any_user_name)


In [20]:
# Function to create FOLLOWS relationship
def create_follows_relationship(tx):
    tx.run("""
        MATCH (a:User {username: 'James'}), (b:User {username: 'Thomas'})
        MERGE (a)-[:FOLLOWS]->(b)
    """)
    print("FOLLOWS relationship created between James and Thomas.")

In [12]:
# Run write transaction
with driver.session() as session:
    session.write_transaction(create_follows_relationship)

/var/folders/mc/wkczfldx7tsb21fbkv6dnnjw0000gn/T/ipykernel_6289/1701395175.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_follows_relationship)


FOLLOWS relationship created between James and Thomas.


In [21]:
def create_relationships(tx):
    tx.run("""
    MATCH (pink:User {username: 'Pink'}),
          (james:User {username: 'James'}),
          (thomas:User {username: 'Thomas'}),
          (you:User {username: 'You'}),
          (casper:User {username: 'Casper'}),
          (lester:User {username: 'Lester'}),
          (elisha:User {username: 'Elisha'}),
          (garfield:User {username: 'Garfield'}),
          (hermann:User {username: 'Hermann'}),
          (odie:User {username: 'Odie'}),
          (jake:User {username: 'Jake'})
    MERGE (pink)-[:FOLLOWS]->(james)
    MERGE (pink)-[:FOLLOWS]->(thomas)
    MERGE (pink)-[:FOLLOWS]->(you)
    MERGE (you)-[:FOLLOWS]->(casper)
    MERGE (you)-[:FOLLOWS]->(lester)
    MERGE (james)-[:FOLLOWS]->(thomas)
    MERGE (thomas)-[:FOLLOWS]->(hermann)
    MERGE (thomas)-[:FOLLOWS]->(jake)
    MERGE (thomas)-[:FOLLOWS]->(garfield)
    MERGE (lester)-[:FOLLOWS]->(elisha)
    MERGE (lester)-[:FOLLOWS]->(garfield)
    MERGE (elisha)-[:FOLLOWS]->(jake)
    MERGE (elisha)-[:FOLLOWS]->(odie)
    MERGE (garfield)-[:FOLLOWS]->(hermann)
    MERGE (hermann)-[:FOLLOWS]->(odie)
    """)

with driver.session() as session:
    session.write_transaction(create_relationships)

print("Relationships created successfully!")


/var/folders/mc/wkczfldx7tsb21fbkv6dnnjw0000gn/T/ipykernel_6289/3411289895.py:32: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationships)


Relationships created successfully!


In [23]:
import re

In [24]:

def get_actor_names(tx):
    result = tx.run("""
        MATCH (a:Person)
        RETURN a.name LIMIT 100
    """)
    return [record["a.name"] for record in result]

with driver.session() as session:
    actor_names = session.read_transaction(get_actor_names)

print("Full Names:", actor_names)

# Use regex to extract lastnames
lastnames = []
for name in actor_names:
    # Assume format "Firstname Lastname" (separated by space)
    match = re.match(r'^\S+\s+(.*)$', name)
    if match:
        lastnames.append(match.group(1))
    else:
        lastnames.append(name)  # fallback: if no space, keep as is

print("Lastnames:", lastnames)

Full Names: ['Keanu Reeves', 'Carrie-Anne Moss', 'Laurence Fishburne', 'Hugo Weaving', 'Andy Wachowski', 'Lana Wachowski', 'Joel Silver', 'Emil Eifrem', 'Charlize Theron', 'Al Pacino', 'Taylor Hackford', 'Tom Cruise', 'Jack Nicholson', 'Demi Moore', 'Kevin Bacon', 'Kiefer Sutherland', 'Noah Wyle', 'Cuba Gooding Jr.', 'Kevin Pollak', 'J.T. Walsh', 'James Marshall', 'Christopher Guest', 'Rob Reiner', 'Aaron Sorkin', 'Kelly McGillis', 'Val Kilmer', 'Anthony Edwards', 'Tom Skerritt', 'Meg Ryan', 'Tony Scott', 'Jim Cash', 'Renee Zellweger', 'Kelly Preston', "Jerry O'Connell", 'Jay Mohr', 'Bonnie Hunt', 'Regina King', 'Jonathan Lipnicki', 'Cameron Crowe', 'River Phoenix', 'Corey Feldman', 'Wil Wheaton', 'John Cusack', 'Marshall Bell', 'Helen Hunt', 'Greg Kinnear', 'James L. Brooks', 'Annabella Sciorra', 'Max von Sydow', 'Werner Herzog', 'Robin Williams', 'Vincent Ward', 'Ethan Hawke', 'Rick Yune', 'James Cromwell', 'Scott Hicks', 'Parker Posey', 'Dave Chappelle', 'Steve Zahn', 'Tom Hanks', '

/var/folders/mc/wkczfldx7tsb21fbkv6dnnjw0000gn/T/ipykernel_6289/431194958.py:9: DeprecationWarning: read_transaction has been renamed to execute_read
  actor_names = session.read_transaction(get_actor_names)


In [25]:
driver.close()